In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os

import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras import backend as K

from PIL import Image
import pandas as pd
import csv
print(tf.__version__)
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
Num GPUs Available:  1


In [2]:
# image processing methods
def process_image(imgtensor):
    imgtensor *= (1/255) #
    return imgtensor

def deprocess_image(imgtensor):
    imgtensor *= 255
    imgtensor = np.clip(imgtensor, 0, 255)
    return imgtensor

train_dir = 'D:/Datasets/VAE_zeroshot/data_punctured_1p25-1_R0p3_RESIZE/train' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
validation_dir = 'D:/Datasets/VAE_zeroshot/data_punctured_1p25-1_R0p3_RESIZE/test' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
hole_dir = 'D:/Datasets/VAE_zeroshot/data_hole_1p25-1_R0p3_RESIZE'
# train_dataframe_dir = 'D:/Datasets/VAE_zeroshot/data_punctured_1p25-1_R0p3_RESIZE/train/category1' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder


In [3]:
#set up training tf.data.Dataset so that sample weights can be given as model input

#load sample weight data
file_CSV = open('data_punctured_weights_train.csv')
data_CSV = csv.reader(file_CSV)
list_CSV = list(data_CSV)
# print(list_CSV[0])
res = [float(i) for i in list_CSV[0]] # convert list of str to list of float
res = np.array(res)
# print(res)

#make dataset that will be zipped to the images dataset to indicate sample weights
weights_dataset = tf.data.Dataset.from_tensor_slices(res)
weights_dataset = weights_dataset.batch(8, drop_remainder=False)
print(weights_dataset.cardinality())

#create images dataset
#set shuffle to False since this dataset must match up with the weights dataset (which isnt shuffled)
img_dataset = image_dataset_from_directory(
    train_dir, shuffle = False, image_size=(256, 256), batch_size=8, color_mode='grayscale', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset = img_dataset.map(process_image)
print(img_dataset.cardinality())

#Proper format for dataset output!: ((input image, sample weight), target output))
#the zipping of these two datasets will create a dataset that generates tuples: (image array (batch), sample weight (batch))
dataset_with_weights_tuple1 = tf.data.Dataset.zip((img_dataset, weights_dataset))
dataset_with_weights_train = tf.data.Dataset.zip((dataset_with_weights_tuple1, img_dataset))

print(dataset_with_weights_train.cardinality())
del weights_dataset, img_dataset, dataset_with_weights_tuple1, res, file_CSV, data_CSV, list_CSV


tf.Tensor(538, shape=(), dtype=int64)
Found 4299 files belonging to 1 classes.
tf.Tensor(538, shape=(), dtype=int64)
tf.Tensor(538, shape=(), dtype=int64)


In [4]:
#set up validation dataset

file_CSV = open('data_punctured_weights_validation.csv')
data_CSV = csv.reader(file_CSV)
list_CSV = list(data_CSV)
# print(list_CSV[0])
res = [float(i) for i in list_CSV[0]] # convert list of str to list of float
res = np.array(res)
# print(res)

#make dataset that will be zipped to the images dataset to indicate sample weights
weights_dataset = tf.data.Dataset.from_tensor_slices(res)
weights_dataset = weights_dataset.batch(8, drop_remainder=False)
print(weights_dataset.cardinality())

#create images dataset
img_dataset = image_dataset_from_directory(
    validation_dir, shuffle = False, image_size=(256, 256), batch_size=8, color_mode='grayscale', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset = img_dataset.map(process_image)
print(img_dataset.cardinality())

#Proper format for dataset output!: ((input image, sample weight), target output))
# dataset_with_weights = img_dataset.map(lambda x: ((x, 1), 0))
#the zipping of these two datasets will create a dataset that generates tuples: (image array (batch), sample weight (batch))
dataset_with_weights_tuple1 = tf.data.Dataset.zip((img_dataset, weights_dataset))
dataset_with_weights_validation = tf.data.Dataset.zip((dataset_with_weights_tuple1, img_dataset))

print(dataset_with_weights_validation.cardinality())
del weights_dataset, img_dataset, dataset_with_weights_tuple1, res, file_CSV, data_CSV, list_CSV

tf.Tensor(27, shape=(), dtype=int64)
Found 211 files belonging to 1 classes.
tf.Tensor(27, shape=(), dtype=int64)
tf.Tensor(27, shape=(), dtype=int64)


In [8]:
#set up "hole" dataset

file_CSV = open('data_hole_weights.csv')
data_CSV = csv.reader(file_CSV)
list_CSV = list(data_CSV)
# print(list_CSV[0])
res = [float(i) for i in list_CSV[0]] # convert list of str to list of float
res = np.array(res)
# print(res)

#make dataset that will be zipped to the images dataset to indicate sample weights
weights_dataset = tf.data.Dataset.from_tensor_slices(res)
weights_dataset = weights_dataset.batch(8, drop_remainder=False)
print(weights_dataset.cardinality())

#create images dataset
img_dataset = image_dataset_from_directory(
    hole_dir, shuffle = False, image_size=(256, 256), batch_size=8, color_mode='grayscale', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset = img_dataset.map(process_image)
print(img_dataset.cardinality())

#Proper format for dataset output!: ((input image, sample weight), target output))
# dataset_with_weights = img_dataset.map(lambda x: ((x, 1), 0))
#the zipping of these two datasets will create a dataset that generates tuples: (image array (batch), sample weight (batch))

dataset_with_weights_tuple1 = tf.data.Dataset.zip((img_dataset, weights_dataset))
dataset_with_weights_hole = tf.data.Dataset.zip((dataset_with_weights_tuple1, img_dataset))

print(dataset_with_weights_hole.cardinality())
del weights_dataset, img_dataset, dataset_with_weights_tuple1, res, file_CSV, data_CSV, list_CSV

tf.Tensor(62, shape=(), dtype=int64)
Found 490 files belonging to 1 classes.
tf.Tensor(62, shape=(), dtype=int64)
tf.Tensor(62, shape=(), dtype=int64)


In [5]:
#Build encoder model
latent_dim = 3 #dimensionality of latent space

encoder_input = Input(shape=(256, 256, 1), name='encoder_input')

x = layers.Conv2D(64, 3, strides=(1, 1), activation='relu', padding='valid', kernel_initializer='RandomNormal',  bias_initializer='zeros')(encoder_input)

x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(64, 3, strides=(1, 1), activation='relu', padding='valid', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)

x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(128, 3, strides=(1, 1), activation='relu', padding='valid', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)

x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(128, 3, strides=(1, 1), activation='relu', padding='valid', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)

x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Flatten(data_format='channels_last')(x)

z_mean = layers.Dense(latent_dim, activation='linear', name='z_mean', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)

## subclass keras layer to sample the latent vector 
class Sample_latent_vector(keras.layers.Layer):
    def sample(self, inputs):
        sigma = 0.1
        z_mean = inputs
        epsilon = K.random_normal(shape=K.shape(z_mean))
        #output is batch of latent vectors
        return z_mean + epsilon*sigma
    
    def call(self, inputs):
        return self.sample(inputs)
    
z = Sample_latent_vector()(z_mean)

# Build Decoder network

x = layers.Dense(16384, activation='relu', use_bias=False, kernel_initializer='RandomNormal')(z)
x = layers.Reshape((32, 32, 16))(x)
x = layers.UpSampling2D(2, name='decoderupsample1')(x)
x = layers.Conv2DTranspose(64, 3, strides=(1, 1), activation='relu', padding='same', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2DTranspose(64, 3, strides=(1, 1), activation='relu', padding='same', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2DTranspose(128, 3, strides=(1, 1), activation='relu', padding='same', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2DTranspose(256, 1, strides=(1, 1), activation='relu', padding='same', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)
x = layers.BatchNormalization(axis=-1)(x)
z_decoded = layers.Conv2DTranspose(1, 1, strides=(1, 1), activation='sigmoid', padding='same', kernel_initializer='RandomNormal',  bias_initializer='zeros')(x)


# make custom loss function by subclassing Layer and using add_loss
class Add_VAE_Loss(keras.layers.Layer):
    
    def compute_VAE_loss(self, inputs):
        sigma = 0.1
        input_image, z_mean, output_image, sample_weights = inputs
        KL_loss = K.mean(K.square(z_mean) + K.exp(K.log(K.square(sigma))) - K.log(K.square(sigma)) - 1, axis=1)
        recon_loss = keras.metrics.binary_crossentropy(K.flatten(input_image), K.flatten(output_image))
        return K.mean((8e-3 * KL_loss + 1*recon_loss)*sample_weights*2, axis=1) #sample weight is provided as an output from the dataset (generator)
        
        
    def call(self, layer_inputs):
        if not isinstance(layer_inputs, list):
            ValueError('Input must be list of [input_image, z_mean, output_image]')
        loss = self.compute_VAE_loss(layer_inputs)
        self.add_loss(loss)
        return layer_inputs[2]
    
#Make VAE model 
outputs = Add_VAE_Loss()([encoder_input, z_mean, z_decoded, sample_weights])
VAE = Model([encoder_input, sample_weights], outputs, name='VAE')
VAE.summary()

VAE.compile(loss=None,
              optimizer=keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics = ['binary_crossentropy'])


Model: "VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 254, 254, 64) 640         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 254, 254, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 127, 127, 64) 0           batch_normalization[0][0]        
________________________________________________________________________________________________

In [6]:
checkpoint_path = r'C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighted_const_variance_Z3D'
#Make checkpoint callback 
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

VAE.fit(dataset_with_weights_train,
        shuffle=True,
        steps_per_epoch=538,#steps_per_epoch needs to equal exactly the number of batches in the Dataset generator
        epochs=10,
        verbose = 2,
        validation_data=dataset_with_weights_validation,
        validation_steps=27,#=#batches in validation dataset
        callbacks=[model_checkpoint_callback])



Epoch 1/10
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighted_const_variance_Z3D\assets
538/538 - 50s - loss: 0.5095 - binary_crossentropy: 0.5120 - val_loss: 0.1765 - val_binary_crossentropy: 0.1591
Epoch 2/10
INFO:tensorflow:Assets written to: C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighted_const_variance_Z3D\assets
538/538 - 50s - loss: 0.1865 - binary_crossentropy: 0.2947 - val_loss: 0.1444 - val_binary_crossentropy: 0.3252
Epoch 3/10
INFO:tensorflow:Assets written to: C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighted_const_variance_Z3D\assets
538/538 - 50s - loss: 0.1265 - binary_crossentropy: 0.2533 - val_loss: 0.1134 - val_binary_crossentropy: 0.2067
Epoch 4/10
INFO:tensorflo

In [9]:
VAE = keras.models.load_model(r"C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighted_const_variance")

# VAE.save_weights('VAE_weighteddata_constvar')
# VAE.save('VAE_model_df')
#Load in weights to VAE. Then copy its latter layers to a new decoder model
# weights_path = r'C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_weighteddata_constvar'
# VAE.load_weights(weights_path)

In [9]:
#PREDICT images 

#visualize image reconstructions
# train_sample = dataset_with_weights_train.take(24)
validation_sample = dataset_with_weights_validation.take(24)
# hole_sample = dataset_with_weights_hole.take(24)

xhat = VAE.predict(validation_sample, steps=3)
# xhat = VAE_2.predict(train_sample, steps=3)
xhat = deprocess_image(xhat)
xhat = xhat.astype(np.uint8)
xhat.shape
for j in range(20):
    pil_im = Image.fromarray(xhat[j,:,:,0],mode='L') # convert each slice of predicted batch to a PIL image object  
    pil_im.save(r'C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\Results\pred_' + f'{j+4300:02}' + '.png')


In [9]:
# make decoder model by copying VAE layers to a new model
VAE.layers[16]
decoder_input = Input(shape = (latent_dim), name='latent_vector')
x = decoder_input
for layer in VAE.layers[16:-2]:
#     print(layer.name)
    x = layer(x)
    
decoder = Model(decoder_input, x, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_vector (InputLayer)   [(None, 32)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16384)             524288    
_________________________________________________________________
reshape (Reshape)            (None, 32, 32, 16)        0         
_________________________________________________________________
decoderupsample1 (UpSampling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 64, 64, 64)        9280      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 128, 128, 64)      0   

In [23]:
#make encoder that skips the sampling layer and just returns z_mean

VAE.layers[14]
detached_encoder_input = Input(shape=(256, 256, 1), name='detached_encoder_input')
x = detached_encoder_input #
for layer in VAE.layers[1:15]:#we must skip the VAE input layer (for loop starts at layer 1 instead of 0) as it causes problems to use the same input layer in two separate models
    x = layer(x)
    
encoder = Model(detached_encoder_input, x, name='encoder')
encoder.summary()

conv2d
batch_normalization
max_pooling2d
conv2d_1
batch_normalization_1
max_pooling2d_1
conv2d_2
batch_normalization_2
max_pooling2d_2
conv2d_3
batch_normalization_3
max_pooling2d_3
flatten
dense
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
detached_encoder_input (Inpu [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 64)      640       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_____________________________________________________________

In [24]:
#create Model for inference which skips the sample_latent_vector layer and just uses z_mean as the latent vector
VAE_2_input = Input(shape=(256, 256, 1), name='VAE_2_input')
VAE_2 = Model(VAE_2_input, decoder(encoder(VAE_2_input)), name="VAE_sampless")
VAE_2.summary()


Model: "VAE_sampless"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
VAE_2_input (InputLayer)     [(None, 256, 256, 1)]     0         
_________________________________________________________________
encoder (Functional)         (None, 32)                1182176   
_________________________________________________________________
decoder (Functional)         (None, 256, 256, 1)       679681    
Total params: 1,861,857
Trainable params: 1,860,065
Non-trainable params: 1,792
_________________________________________________________________
